# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.core import Workspace, Experiment


import os

In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:

experiment_name = 'hyperdriverun'

experiment=Experiment(ws, experiment_name)

In [ ]:
#Create compute cluster
amlcompute_cluster_name="runhyperdrive"

#verify that cluster does not already exist
try:
    aml_compute=ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config=AmlCompute.provisioning_configuration(vm_size="Standard_D3_V2",max_nodes=4)
    aml_compute=ComputeTarget.create(ws,amlcompute_cluster_name,compute_config)

aml_compute.wait_for_completion(show_output=True)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
#Define RunConfig for the compute
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive.parameter_expressions import choice
import os

# Specify parameter sampler (continuous)
ps = RandomParameterSampling ({
    "--C":uniform(0.05,1.0), #equivalent to the value of k
    "--max_iter":choice(10,25,50,75,100)
    #max_iter between 1 to 999
})

# Specify a Policy
early_termination_policy = BanditPolicy(slack_factor=0.1,evaluation_interval=1,delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='.',
compute_target=aml_compute,vm_size="Standard_D3_V2",vm_priority="lowpriority",entry_script="training.py")

In [ ]:
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
hyperdrive_config = HyperDriveConfig(
    estimator=est,
    hyperparameter_sampling=ps,
    policy=early_termination_policy,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=30,
    max_concurrent_runs=5
)

In [ ]:
#TODO: Submit your experiment
from azureml.widgets import RunDetails
hyperdrive_run=exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(hyperdrive_run).show()

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
import joblib
# Get your best run and save the model from that run.
best_run=hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics=best_run.get_metrics()
parameter_values=best_run.get_details()['runDefinition']['arguments']

print('Best Run ID:',best_run.id)
print('\n Accuracy:',best_run_metrics['Accuracy'])

In [ ]:

print(best_run)
best_run

In [ ]:
#TODO: Save the best model
model=best_run.register_model(model_name='hyperdrive_model',model_path='./outputs/model.joblib')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service